In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install PyPDF2

In [ ]:
import os
import json
import time
import base64
import PyPDF2
import io
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# путь к папке с html файлами
source = os.path.join(os.getcwd(), 'html_variants')
# путь к папке для сохранения результатов
output = os.path.join(os.getcwd(), 'pdf_variants')
# шаг, с которым мы подбираем масштаб страницы
scale_step = 0.025

In [ ]:
def send_devtools(driver, cmd, params={}):
    resource = "/session/%s/chromium/send_command_and_get_result" % driver.session_id
    url = driver.command_executor._url + resource
    body = json.dumps({'cmd': cmd, 'params': params})
    response = driver.command_executor._request('POST', url, body)
    if response.get('status'):
        raise Exception(response.get('value'))
    return response.get('value')

def latex_is_loaded(arg):
    state = driver.execute_script('return window.latexIsLoaded;')
    #print('latex_state=%s' % state)
    return state == True

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for root, dirs, files in os.walk(source):
    for filename in files:
        if not filename.endswith(".html"):
            continue
            
        # открываем html файл
        print('Открываем файл:', filename)
        driver.get(f'file://{os.path.join(os.getcwd(), root, filename)}')
        
        print('Дожидаемся полной инициализации...')
        
        # дожидаемся загрузки страницы
        element = WebDriverWait(driver, 10).until(ec.presence_of_element_located((By.TAG_NAME, 'html')))
        
        # дожидаемся полной инициализации LaTex на странице
        driver.execute_script('window.latexIsLoaded = false;')
        driver.execute_script('window.MathJax.Hub.Register.StartupHook("End",function () { window.latexIsLoaded = true; });')
        WebDriverWait(driver, 10).until(latex_is_loaded)
        
        # получаем pdf
        
        last_num_pages = -1
        last_pdf_file = None
            
        print_options = {
            'scale': 1,
        }
        
        print('Подбираем коэффициент масштабирования...')
        while 0 < print_options['scale'] < 2:
            result = send_devtools(driver, "Page.printToPDF", print_options)
            result = base64.b64decode(result['data'])

            pdf_file = io.BytesIO(result)
            pdf_reader = PyPDF2.PdfFileReader(pdf_file)
            num_pages = pdf_reader.numPages
            #print('При масштабировании {} кол-во страниц: {}'.format(print_options['scale'], num_pages))
            
            if last_num_pages > 1 and num_pages == 1:
                break
            elif last_num_pages == 1 and num_pages == 2:
                result = last_pdf_file
                break
            elif num_pages == 1:
                print_options['scale'] += scale_step
            else:
                print_options['scale'] -= scale_step
                  
            last_num_pages = num_pages
            last_pdf_file = result
        
        
        # сохраняем pdf
        print('Сохраняем файл...', '\n')
        with open(os.path.join(output, os.path.splitext(filename)[0] + '.pdf'), 'wb') as file:
            file.write(result)
        
driver.close()
print('Готово!')